In [1]:
import wget

In [ ]:
#wget.download('http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/All_Beauty.json.gz')

In [2]:
import pandas as pd
import gzip
import json

data = []
with gzip.open('All_Beauty_5.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()).get('reviewText'))

print(len(data))

print(data[0])

data = [x for x in data if x!=None]

print(len(data))

5269
As advertised. Reasonably priced
5264


В чём, собственно, проблема, о наличии которой можно подумать, увидев это высказывание -- в отзывы на конкретный продукт сам продукт можно и не упоминать

# 1. Как найти товары: способы

*(3 балла) Предложите 3 способа найти упоминания товаров в отзывах. Например, использовать bootstrapping: составить шаблоны вида "холодильник XXX", найти все соответствующие n-граммы и выделить из них называние товара. Могут помочь заголовки и дополнительные данные с Amazon (Metadata здесь) Какие данные необходимы для каждого из способов? Какие есть достоинства/недостатки?*

__1. Читать и размечать, как, например, именованные сущности. После определённого количества размеченного можно попробовать обучить модель.__
 <br>
Плюсы: точно; не надо быть лингвистом; можно отдать на Толоку<br>
Минусы: долго и трудозатртно; можеть быть дорого; может получиться далеко не так точно, как предполагалось, из-за несогласия разметчиков или неясности правил<br>
 <br>
__2. Сложить всё в ворд-ту-век, надеяться на то, что упоминания товаров окажутся там в одном кластере__<br>
 <br>
Плюсы: Если повезёт, можно сгруппировать упоминания одного товара/синонимы. Относительно быстро. Не надо привлекать людей<br>
Минусы: Упоминания товаров часто не однословны, нужно что-то с биграммами придумывать. Или с триграммами. <br>
 <br>
__3. Привлечь тяжёлую артиллерию, а именно млодели для извлечения именованных сущностей__<br>
 <br>
Плюсы: Есть вероятность, что получится неплохая точность. Можно расширять список за счёт применения новых и новых идей<br>
Минусы: сложно, получится, скорее всего, не идеально
 <br>

# Извлечение названий товаров

*(2 балла) Реализуйте один из предложенных вами способов.*

Ну что ж, попробуем. Посмотрим на наши данные!

In [ ]:
data[30:50]

Здесь выделяются три типа упоминаний. <br>
1. Никак не упоминаем продукт или называем его "it"
2. this NOUN, ADJ + NOUN
3. Собственно названия
 <br>
 
И что же мы будем с этим делать? Ну, давайте действовать в два этапа. Даже в три. Пункт первый -- постараться извлечь именованные сущности и посмотреть на них

### Достаём именованные сущности

Stanza очень хороша! Но при этом она работает достаточно долго. 

In [66]:
import stanza
from tqdm import tqdm
#stanza.download('en')
nlp = stanza.Pipeline(lang='en', processors='tokenize,ner')
def stanza_result(text, nlp=nlp):
    doc = nlp(text)
    ent_texts = [ent.text for sent in doc.sentences for ent in sent.ents]
    ent_types = [ent.type for sent in doc.sentences for ent in sent.ents]
    #print(*[f'entity: {ent.text}\ttype: {ent.type}' for sent in doc.sentences for ent in sent.ents], sep='\n')
    return ent_texts, ent_types

2021-12-07 04:38:56 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| ner       | ontonotes |

2021-12-07 04:38:56 INFO: Use device: gpu
2021-12-07 04:38:56 INFO: Loading: tokenize
2021-12-07 04:38:56 INFO: Loading: ner
2021-12-07 04:38:56 INFO: Done loading processors!


In [67]:
ent_texts = [] 
ent_types = []
for el in tqdm(data):
    #print(i)
    a = stanza_result(el)
    ent_texts.extend(a[0])
    ent_types.extend(a[1])

 89%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉              | 4693/5264 [06:35<00:48, 11.87it/s]


RuntimeError: CUDA out of memory. Tried to allocate 72.00 MiB (GPU 0; 2.00 GiB total capacity; 680.95 MiB already allocated; 0 bytes free; 728.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
products = []
for i, el in enumerate(ent_types):
    if el == 'PRODUCT':
        products.append(i)
        print(ent_texts[i])

In [ ]:
len(products)

### Часть вторая

Во-первых, stanza достатчно хороша в доставании названий продуктов. Даже очень хороша. Временами она достаёт названия фирм или компаний -- но во многих случаях это довольно логично: многие упоминания выглядят, как "шампунь X". А нам такое не очень нравится, наверное, потому что мы, вероятно, хотим знать, какой именно это продукт. Поэтому давайте экспериментировать с морфологией и отзывами.<br>
 <br>
Можно сразу начать думать обо всех пунктах задания. Что нам, в общем-то, нужно? Нам нужны упоминания товаров. Желательно всех товаров, которые упоминаются в отзыве, а он там может быть не один. Чтобы такое добыть и, например, сформировать хорошие N-граммы с сущностями, по которым потом тоже можно искать, можно посмотреть на то, как наши упоминания распределяются по отзывам.

(1 балл) Соберите n-граммы с полученными сущностями (NE + левый сосед / NE + правый сосед)<br>
 <br>
(3 балла) Ранжируйте n-граммы с помощью 3 коллокационных метрик (t-score, PMI и т.д.). Не забудьте про частотный фильтр / сглаживание. Выберите лучший результат (какая метрика ранжирует выше коллокации, подходящие для отчёта).<br>
 <br>
(1 балл) Сгруппируйте полученные коллокации по NE, выведите примеры для 5 товаров. Должны получиться примерно такие группы:<br>
 <br>
(2 балла): если придумаете способ объединить синонимичные упоминания (например, "Samsung Galaxy Watch", "watch", "smartwatch")

In [3]:
import stanza
from tqdm import tqdm
#stanza.download('en')
nlp = stanza.Pipeline(lang='en', processors='tokenize,lemma,pos,ner')

2021-12-07 00:12:26 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| ner       | ontonotes |

2021-12-07 00:12:26 INFO: Use device: gpu
2021-12-07 00:12:26 INFO: Loading: tokenize
2021-12-07 00:12:28 INFO: Loading: pos
2021-12-07 00:12:28 INFO: Loading: lemma
2021-12-07 00:12:28 INFO: Loading: ner
2021-12-07 00:12:28 INFO: Done loading processors!


In [4]:
def stanza_doc(text, nlp=nlp):
    doc = nlp(text)
    return doc

Берём ещё организации, чтобы было полнее -- stanza распознаёт часть упоминаний вроде "продукт Шанель" как "организация Шанель". В препроцессинге происходит удаление всех предлогов, собзов, артиклей -- всё то, что в школе бы назвали служебными частями речи. Здесь одновременно происходит добывание упоминаний и собирание n-грамм

In [32]:
def stanza_result(doc):
    ent_texts = [ent.text for sent in doc.sentences for ent in sent.ents]
    ent_types = [ent.type for sent in doc.sentences for ent in sent.ents]
    ent_ngrams = []
    this_list_1 = []
    this_list_2 = []
    one_ngram = []
    stops = ['ADP', 'AUX', 'CCONJ', 'DET', 'INTJ', 'PART', 'PRON', 'PUNCT', 'SCONJ']
    for sent in doc.sentences:
        #list_tokens = list(token for token in sent.tokens if token.upos != 'PUNCT')
        list_tokens = list(token for token in sent.tokens if token.words[0].upos not in stops)
        for i, token in enumerate(list_tokens):
            if token.words[0].lemma == 'this':
                if i + 1 < len(list_tokens):
                    this_list_1.append(list_tokens[i+1])
                    if i + 2 < len(list_tokens):
                        this_list_2.append(list_tokens[i+2])
        #print(sent.ents)
            if token.ner == 'B-ORG' or token.ner == 'B-PRODUCT':
                ent_ngrams.append(one_ngram)
                one_ngram = []
                if i > 0:
                    one_ngram.append(list_tokens[i-1])
                else:
                    one_ngram.append('START')
                one_ngram.append(token)
            elif token.ner == 'I-ORG' or token.ner == 'I-PRODUCT':
                one_ngram.append(token)
            elif token.ner == 'E-ORG' or token.ner == 'E-PRODUCT':
                one_ngram.append(token)
                if i + 1 < len(list_tokens):
                    one_ngram.append(list_tokens[i+1])
                else:
                    one_ngram.append('END')
            elif token.ner == 'S-ORG' or token.ner == 'S-PRODUCT':
                ent_ngrams.append(one_ngram)
                one_ngram = []
                if i > 0:
                    one_ngram.append(list_tokens[i-1])
                else:
                    one_ngram.append('START')
                one_ngram.append(token)
                if i + 1 < len(list_tokens):
                    one_ngram.append(list_tokens[i+1])
                else:
                    one_ngram.append('END')
        
    ent_ngrams.append(one_ngram)

            
    
    
    #print(*[f'entity: {ent.text}\ttype: {ent.type}' for sent in doc.sentences for ent in sent.ents], sep='\n')
    return ent_texts, ent_types, ent_ngrams, this_list_1, this_list_2 

In [30]:
stanza_docs = []
for text in tqdm(data):
    stanza_docs.append(stanza_doc(text, nlp=nlp))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5264/5264 [11:42<00:00,  7.49it/s]


In [33]:
%%time
all_ent_texts = []
all_ent_types = []
all_ent_ngrams = []
all_this_list_1 = []
all_this_list_2 = []

for text in tqdm(data):
    ent_texts, ent_types, ent_ngrams, this_list_1, this_list_2 = stanza_result(stanza_doc(text, nlp=nlp))
    all_ent_texts.append(ent_texts)
    all_ent_types.append(ent_types)
    all_ent_ngrams.append(ent_ngrams)
    all_this_list_1.append(this_list_1)
    all_this_list_2.append(this_list_2)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5264/5264 [11:38<00:00,  7.54it/s]

Wall time: 11min 38s


In [35]:
all_text = []
stops = ['ADP', 'AUX', 'CCONJ', 'DET', 'INTJ', 'PART', 'PRON', 'PUNCT', 'SCONJ']
for one_doc in stanza_docs:
    for sent in one_doc.sentences:
        all_text.extend([word.text.lower() for word in sent.words if word.upos not in stops])

## Считаем метрики

In [68]:
def count_freq(our_ngram, all_texts):
    stops = ['START', 'END']
    left_context = len([i for i in all_text if i == our_ngram[0] and not i in stops])
    right_context = len([i for i in  all_text if i == our_ngram[-1] and not i in stops])
    our_ent_freq = 0
    our_ent = our_ngram[1:-1]
    
    index_first = []
    index_second = []
    if len(our_ent) == 1:
        our_ent = our_ent[0]
        for word in all_text:
            if our_ent == word:
                our_ent_freq += 1
    
    else:
        for i, word in enumerate(all_texts):
            if word == our_ent[0]:
                index_first.append(i)
                
        for i, word in enumerate(our_ent[1:]):
            for el in index_first:
                if all_texts[el + i + 1] == word:
                    index_second.append(el)
            index_first = index_second
            index_second = []
        our_ent_freq = len(index_first)
        
    return left_context, right_context, our_ent_freq

In [69]:
def count_freq_ngram(our_ngram, all_texts):
    index_first = []
    index_second = []    
    for i, word in enumerate(all_texts):
        if word == our_ngram[0]:
            index_first.append(i)

    for i, word in enumerate(our_ngram[1:]):
        for el in index_first:
            if all_texts[el + i + 1] == word:
                index_second.append(el)
        index_first = index_second
        index_second = []
        
    return len(index_first)

In [38]:
all_word_ngrams = []

for one_text in all_ent_ngrams:
    for one_ngram in one_text:
        one_ngram_words = []
        for word in one_ngram:
            if word == 'START' or word == 'END':
                 one_ngram_words.append(word)
            else:
                one_ngram_words.append(word.text)
        if one_ngram_words != []:
            all_word_ngrams.append(one_ngram_words)

In [39]:
all_word_ngrams

[['Williams', 'Company', 'ran'],
 ['is', 'AQUA', 'VELVA', 'MAN', 'seemed'],
 ['negative', 'Aqua', 'Velva', 'last'],
 ['have', 'Ambergris', 'quite'],
 ['one', 'Wal', 'Mart', 'glass'],
 ['START', 'Selenium', 'appeared'],
 ['get', 'Amazon', 'END'],
 ['have', 'Airbnb', 'END'],
 ['Dave', 'Buster'],
 ['START', 'Uber', 'pleased', 'D&G', 'Lifht', 'Blue', 'one'],
 ['pleased', 'Uhuru', 'Naturals', 'Hair'],
 ['START', 'Google', 'uses'],
 ['well', 'Northern', 'Fir'],
 ['START', 'Eau', 'de', 'Hadrien', 'favorite'],
 ['Tangle', 'Taming', 'Conditioner', 'good'],
 ['tried', 'Kerastase', 'PHYTO'],
 ['Kerastase', 'PHYTO', 'Davines'],
 ['wish', 'Redken', 'discontinued'],
 ['Federal',
  'Trade',
  'Commission',
  'Guidelines',
  'Testimony',
  'Advertising',
  'END'],
 ['ago', 'Sally', 'Beauty', 'Supply', 'END'],
 ['PRELL', 'Conditioner', 'far'],
 ['company', 'Amazon', 'buy'],
 ['discontinued', 'Bain', 'De', 'Terre', 'END'],
 ['used', 'Mousse', 'products'],
 ['START', 'Wish', 'Walgreens', 'kept'],
 ['use'

In [40]:
import math

In [70]:
def count_metrics(our_ngram, all_text):
    freq_of_bigram = count_freq_ngram(our_ngram, all_text)
    left_context, center, right_context = count_freq(our_ngram, all_text)
    num_bigrams = len(all_text) - 2
    #было бы сложно иным образом оценить количество биграмм -- с одной стороны, у нас есть многоcловные успоминания,
    #с другой -- там также могут быть фамилии и т.д. Это примерное число именно возможных биграмм, и оно не сильно 
    #отдаляется от настоящего числа биграмм на нашем довольно большом корпусе, кроме того, оно для всех одинаково
    if freq_of_bigram != 0 and left_context != 0 and right_context != 0 and center != 0:
        t_score_left = (freq_of_bigram - (left_context * center)/num_bigrams)/(freq_of_bigram**(1/2))
        t_score_right = (freq_of_bigram - (right_context * center)/num_bigrams)/(freq_of_bigram**(1/2))
        dice_left = (2*freq_of_bigram)/(left_context + center)
        dice_right = (2*freq_of_bigram)/(right_context + center)
        pmi_left = math.log2(freq_of_bigram / (left_context * center))
        pmi_right = math.log2(freq_of_bigram / (right_context * center))
        
    else:
        t_score_left, t_score_right, dice_left, dice_right, pmi_left, pmi_right = [0, 0, 0, 0, 0, 0]
    
    return t_score_left, t_score_right, dice_left, dice_right, pmi_left, pmi_right

In [71]:
t_scores_left, t_scores_right, dices_left, dices_right, pmis_left, pmis_right = [], [], [], [], [], []

for ngram in tqdm(all_word_ngrams):
    ngram = [word.lower() for word in ngram]
    if len(ngram) > 2:
        t_score_left, t_score_right, dice_left, dice_right, pmi_left, pmi_right = count_metrics(ngram, all_text)
        t_scores_left.append(t_score_left)
        t_scores_right.append(t_score_right)
        dices_left.append(dice_left)
        dices_right.append(dice_right)
        pmis_left.append(pmi_left)
        pmis_right.append(pmi_right)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1415/1415 [00:30<00:00, 46.97it/s]


In [97]:
import numpy as np
a = np.argsort(t_scores_left)[::-1][:20]
for i in a:
    print(all_word_ngrams[i])

['prolong', 'Spongelle', 'useful']
['used', 'Bellmira', 'Herbaflor', 'Herbal', 'Baths', 'years']
['see', 'P&G', 'bring']
['body', 'Spongelle', 'END']
['get', 'Amazon', 'END']
['START', 'Crest', 'Pro-Health', 'Life', 'helped']
['Ethylhexylglycerin', 'Tocopherol', 'Limonene']
['body', 'Spongelle', 'END']
['gel', 'Solstis', 'really']
['get', 'Amazon', 'END']
['used', 'Bellmira', 'Herbaflor', 'Herbal', 'Baths', 'years']
['Thanks', 'Amazon', 'no']
['START', 'Eau', 'de', 'Hadrien', 'favorite']
['get', 'Amazon', 'END']
['START', 'Google', 'uses']
['used', 'WaterPik', 'great']
['tried', 'Kerastase', 'PHYTO']
['shampoo', 'Detangler', 'Conditioner']
['product', 'Sally', 'Beauty', 'Supply', 'END']
['Shower', 'Gel', 'hoping']


In [96]:
a = np.argsort(t_scores_right)[::-1][:20]
for i in a:
    print(all_word_ngrams[i])

['prolong', 'Spongelle', 'useful']
['get', 'Amazon', 'END']
['see', 'P&G', 'bring']
['used', 'Bellmira', 'Herbaflor', 'Herbal', 'Baths', 'years']
['body', 'Spongelle', 'END']
['START', 'Eau', 'de', 'Hadrien', 'favorite']
['used', 'WaterPik', 'great']
['Thanks', 'Amazon', 'no']
['tried', 'Kerastase', 'PHYTO']
['get', 'Amazon', 'END']
['START', 'Google', 'uses']
['used', 'Bellmira', 'Herbaflor', 'Herbal', 'Baths', 'years']
['body', 'Spongelle', 'END']
['Ethylhexylglycerin', 'Tocopherol', 'Limonene']
['get', 'Amazon', 'END']
['gel', 'Solstis', 'really']
['START', 'Crest', 'Pro-Health', 'Life', 'helped']
['START', 'Crest', 'Pro-Health', 'Life', 'helped']
['whey', 'Old', 'Spice', 'discontinued']
['Tangle', 'Taming', 'Conditioner', 'good']


In [95]:
a = np.argsort(dices_left)[::-1][:20]
for i in a:
    print(all_word_ngrams[i])

['always', 'Bare', 'Minerals', 'never']
['shipped', 'FPO', 'address']
['Oil', 'Glycol', 'Distearate', 'Fragrance']
['Distearate', 'Fragrance', 'Organic']
['Chamomile', 'Althea', 'Sage']
['Axe', 'Detailer', 'Shower', 'Tool', 'basically', 'Ax', 'Shower', 'Tool', 'get']
['consider', 'Macallan', 'single']
['turned', 'Amazon', 'END']
['looking', 'Lectric', 'Shave', 'END']
['made', 'Gillette', 'St.']
['CMIT', 'FD&C', 'Blue', '1', 'Red']
['Chamomile', 'Althea', 'Sage']
['tried', 'Essie', 'polish']
['Glycerin', 'Potassium', 'Hydroxide', 'Organic']
['Hydroxide', 'Organic', 'Coconut', 'Oil', 'Glycol']
['g', 'Traditional', 'Fragrance', 'Sandalwood', '/']
['Bvlgari', 'Conditioner', 'favorite']
['turned', 'Amazon', 'END']
['Distearate', 'Fragrance', 'Organic']
['Get', 'Grip', 'Trio', 'Manicure', 'Pedicure']


In [94]:
a = np.argsort(dices_right)[::-1][:20]
for i in a:
    print(all_word_ngrams[i])

['Alcohol', 'Pectin', 'Papaya']
['single', 'Escada', 'fragrance']
['Glycerin', 'Potassium', 'Hydroxide', 'Organic']
['Oil', 'Cetearyl', 'Alcohol', 'Pectin']
['points', 'QC', 'issue']
['Oil', 'Glycol', 'Distearate', 'Fragrance']
['use', 'Estee', 'Lauder', 'foundation']
['Hydroxide', 'Organic', 'Coconut', 'Oil', 'Glycol']
['Urban', 'Spa', 'Microfiber', 'Bath', 'Pillow', 'makes']
['Water', 'Stearic', 'Acid', 'Myristic']
['consider', 'Macallan', 'single']
['tried', 'Essie', 'polish']
['original', 'Gillette', 'equipment']
['Hydroxide', 'Organic', 'Coconut', 'Oil', 'Glycol']
['START', "Sam's", 'Club', 'member']
['Lectric', 'Shave', 'END']
['Get', 'Grip', 'Trio', 'Manicure', 'Pedicure']
['get', 'Crest', 'pro']
['Lectric', 'Shave', 'END']
['Alcohol', '40', 'B', 'Bergamot']


In [98]:
a = np.argsort(pmis_left)[::-1][:20]
for i in a:
    print(all_word_ngrams[i])

['other', 'Crest', 'Pro-Heath', 'products']
['Braun', '320s', '4', '#']
['Ethylhexyl', 'Octyl', 'Palmitate', 'Ethylhexylglycerin']
['Ethylenediamine', 'Ethylhexyl', 'Octyl']
['Phenoxyethanol', 'Tetrahydroxypropyl', 'Ethylenediamine', 'Ethylhexyl']
['Fragrance', 'Organic', 'Jojoba', 'Seed', 'Oil', 'Organic', 'Pomegranate', 'Oil', 'Organic', 'Green', 'Tea', 'Leaf', 'Extract', 'Organic', 'Shea', 'Butter', 'Fruit', 'Phenoxyethanol']
['brush', 'Urban', 'Spa', 'END']
['adherence', 'FTC', 'regulations']
['ms-3/4700', 'Remington', 'micro']
['store', "Norelco's", 'one']
['have', 'Airbnb', 'END']
['review', 'Sea', 'Salt', 'w', 'Kelp', 'soap']
['shower', 'Gel', 'END']
['However', 'Tea', 'Tree', 'often']
['day', 'SPA', 'LOVE']
['buy', 'Kerastase', 'Fusio-', 'Dose', 'Systeme', 'Professionnel', 'END']
['fan', 'Molton', 'Brown', 'products']
['day', 'SPA', 'LOVE']
['Palmitate', 'Ethylhexylglycerin', 'Tocopherol']
['see', 'P&G', 'bring']


In [102]:
a = np.argsort(pmis_right)[::-1][:20]
for i in a:
    print(all_word_ngrams[i])

['other', 'Crest', 'Pro-Heath', 'products']
['again', 'Amazon', 'thanks']
['Palmitate', 'Ethylhexylglycerin', 'Tocopherol']
['Ethylhexyl', 'Octyl', 'Palmitate', 'Ethylhexylglycerin']
['Ethylenediamine', 'Ethylhexyl', 'Octyl']
['Phenoxyethanol', 'Tetrahydroxypropyl', 'Ethylenediamine', 'Ethylhexyl']
['Fragrance', 'Organic', 'Jojoba', 'Seed', 'Oil', 'Organic', 'Pomegranate', 'Oil', 'Organic', 'Green', 'Tea', 'Leaf', 'Extract', 'Organic', 'Shea', 'Butter', 'Fruit', 'Phenoxyethanol']
['brush', 'Urban', 'Spa', 'END']
['adherence', 'FTC', 'regulations']
['ms-3/4700', 'Remington', 'micro']
['Braun', '320s', '4', '#']
['store', "Norelco's", 'one']
['review', 'Sea', 'Salt', 'w', 'Kelp', 'soap']
['shower', 'Gel', 'END']
['However', 'Tea', 'Tree', 'often']
['day', 'SPA', 'LOVE']
['buy', 'Kerastase', 'Fusio-', 'Dose', 'Systeme', 'Professionnel', 'END']
['fan', 'Molton', 'Brown', 'products']
['day', 'SPA', 'LOVE']
['have', 'Airbnb', 'END']


Метрика DICE с левым контекстом больше всех похожа на правду -- там больше всего названий продуктов. Без пунктуации и служебных частей речи всё выглядит гораздо более осмысленно. Со служебными частями речи левый контекст работал плохо, потому что там в английском предлоги, союзы и артикли. С включенной очисткой он стал более информативным, а в правый контекст попало больше составов, чем названий.

## Выводим тип продуктов

In [65]:
for word in ['shampoo', 'conditioner', 'soap', 'butter', 'lotion']:
    print('***************')
    print(word)
    print('***************')
    for ngram in set(' '.join(i) for i in all_word_ngrams):
        if word in ngram.lower():
            print(ngram)

***************
shampoo
***************
shampoo Pantene does
shampoo Detangler Conditioner
like H20 shampoo
START Volume Conditioner shampoo
classic Pantene Shampoo END
original Matrix Amplifying / Volumizing System Shampoo number
liked Pantene shampoo
***************
conditioner
***************
conditioner Pantene END
PRELL Conditioner far
use Matrix conditioner
shampoo Detangler Conditioner
START Volume Conditioner shampoo
START Matrix Biolage Fortifying conditioner
Tangle Taming Conditioner good
START Fekkai Foam Conditioner unusual
Bvlgari Conditioner favorite
Blown Aerosol Foam Conditioner conditioner
***************
soap
***************
soap Kirkland body
START Pre de Provence Shea Butter Enriched Artisanal French Soap Bar good
supply Pre de Provence Soap Milk
Soap Milk fantastic
Artisanal French Soap Bar has
bar Mint Leaf Pre de Provence Shea Butter Soap delightful
fourth Pre de Provence soap
bar Pre de Provence Butter Soap Violette
loved Pre de Provence soaps
START Pre de Prove

Вот так гораздо лучше! Без служебных слов всё становится более осмысленным и информативным